In [1]:
#Import all the required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys 
import os
import tensorflow as tf
import pickle
import sklearn
import itertools
%matplotlib inline

from sklearn.model_selection import train_test_split 
from sklearn import datasets 
from tensorflow import keras
from tensorflow.keras import models, layers
from keras.layers import RepeatVector, Input, Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, MaxPool2D
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.models import Sequential
from sklearn import preprocessing
from keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Concatenate, Reshape
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix 
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from keras.layers import BatchNormalization
from sklearn.model_selection import KFold
import warnings

warnings.filterwarnings("ignore")


In [2]:
data_path = "BALANCED DATASET 3/BALANCE DATASET 3"
IMG_SIZE = (256, 256)
BATCH_SIZE = 32
NUM_EPOCHS = 7
NUM_CLASSES = 4


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    seed=42)

val_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    seed=42)

test_generator = test_datagen.flow_from_directory(
    data_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)


Found 2824 images belonging to 4 classes.
Found 704 images belonging to 4 classes.
Found 3528 images belonging to 4 classes.


In [4]:
model_input = Input(shape=(256, 256, 3))

In [5]:
input_shape = (256, 256, 3) # Change to (512,512,3)
input_tensor = model_input

# create the VGG model
vgg_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

# freeze the first few layers
for layer in vgg_model.layers[:10]:
   layer.trainable = False

# create the Xception model
xception_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

# freeze the first few layers
for layer in xception_model.layers[:10]:
   layer.trainable = False

# concatenate the output tensors
vgg_output = vgg_model(input_tensor)
xception_output = xception_model(input_tensor)
concatenated = tf.keras.layers.Concatenate(axis=-1)([vgg_output, xception_output])

# add some more layers
x = tf.keras.layers.GlobalMaxPooling2D()(concatenated)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
classification_layer = tf.keras.layers.Dense(4, activation='sigmoid')(x)

# create the final model
ensemble_model = tf.keras.models.Model(inputs=input_tensor, outputs=classification_layer)

# compile the model
ensemble_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = ensemble_model.fit(
    train_generator,
    batch_size=BATCH_SIZE,
    validation_data=val_generator,
    verbose=1,
    epochs=NUM_EPOCHS,
)


Epoch 1/7
89/89 [==============================] - 4508s 51s/step - loss: 0.4437 - accuracy: 0.5786 - val_loss: 0.3676 - val_accuracy: 0.6506
Epoch 2/7
89/89 [==============================] - 7132s 80s/step - loss: 0.2603 - accuracy: 0.7918 - val_loss: 0.2858 - val_accuracy: 0.7287
Epoch 3/7
89/89 [==============================] - 4384s 49s/step - loss: 0.1770 - accuracy: 0.8697 - val_loss: 0.3531 - val_accuracy: 0.7145
Epoch 4/7
79/89 [=========================>....] - ETA: 8:33 - loss: 0.1342 - accuracy: 0.8970

In [ ]:
loss_val, acc_val = ensemble_model.evaluate(test_generator, verbose=1)
print("Test: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))


In [ ]:
loss_val, acc_val = ensemble_model.evaluate(val_generator, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))


In [ ]:
yTrue = test_generator.classes

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

classes_ = ['Acne', 'Eczema', 'Melanoma', 'Psoriasis']

# Generate the confusion matrix for the model
y_pred = np.argmax(ensemble_model.predict(test_generator), axis=-1)
conf_mat = confusion_matrix(yTrue, y_pred, labels=range(len(classes_)))

group_names = ['True Acne','False Eczema','False Melanoma','False Psoriasis',
               'False Acne','True Eczema','False Melanoma','False Psoriasis',
               'False Acne','False Eczema','True Melanoma','False Psoriasis',
               'False Acne','False Eczema','False Melanoma','True Psoriasis']

group_counts = ["{0:0.0f}".format(value) for value in
                conf_mat.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     conf_mat.flatten()/np.sum(conf_mat)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names, group_counts, group_percentages)]
labels = np.asarray(labels).reshape(len(classes_),len(classes_))

ax = sns.heatmap(conf_mat, annot=labels, yticklabels=classes_, xticklabels=classes_, fmt='', cmap='OrRd')
ax.set(title='Confusion Matrix', xlabel='Predicted Diseases', ylabel='Actual Diseases')
ax.plot()

In [ ]:
#function to evaluate models
def evaluate(y_pred):
    print(f"Accuracy: \n{accuracy_score(test_generator,y_pred)}\n")
    print(f"Classification report: \n{classification_report(test_generator,y_pred)}\n")
    cm = confusion_matrix(test_generator,y_pred)
    
    class_names = ['True Acne','True Eczema','True Melanoma','True Psoriasis'] 
    sns.heatmap(conf_mat, annot=labels, yticklabels=classes_, xticklabels=classes_, fmt='', cmap='OrRd')
    plt.xlabel('Predicted Diseases')
    plt.ylabel('Actual Diseases')
    plt.title('Confuson matrix')  
    plt.show()

In [ ]:
#yTrue = test_generator.classes

In [ ]:
report = classification_report(yTrue, y_pred, labels=range(len(classes_)))
print(report)

In [ ]:
accuracy = history.history['accuracy']
validationAccuracy = history.history['val_accuracy']

loss = history.history['loss']
validationLoss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(NUM_EPOCHS), accuracy, label='Training Accuracy')
plt.plot(range(NUM_EPOCHS), validationAccuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(NUM_EPOCHS), loss, label='Training Loss')
plt.plot(range(NUM_EPOCHS), validationLoss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#Confusion matrix graphh
confusionMatrix = confusion_matrix(yTrue, yPrediction)
sns.heatmap(confusionMatrix, annot=True, cmap="Blues")
plt.show

In [ ]:
# true labels for the test data
yTrue = yTrue

# Get the class names
class_names = list(val_generator.class_indices.keys())

# classification report
clsreport = classification_report(yTrue, yPrediction, target_names=class_names)
print(clsreport)

In [ ]:
filename = "modelEnsemble.pkl"
with open(filename,'wb') as f:
    pickle.dump(ensemble_model,f)